This notebook compares the performance of an AutoGluon-generated model with both a normal PyTorch model and a Quantization-Aware Training (QAT) PyTorch model.
It performs the following steps:
1. Load the dataset (train, test, and validation sets).
2. Load the AutoGluon predictor.
3. Load the replicated normal PyTorch model and the QAT PyTorch model.
4. Evaluate and compare the accuracy and log loss of all models.
5. Compare the complexity (number of parameters) and size of all models.
6. Visualize the results with plots.

Make sure to update the paths to the models and dataset accordingly.


Import necessary libraries

In [2]:

import os
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from autogluon.tabular import TabularPredictor, TabularDataset
import matplotlib.pyplot as plt

/u/carloshh/tiny_model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Paths, update if needed

In [1]:
predictor_path = '/path/to/your/predictor'  # Update this path as necessary
data_dir = './datasets/CICIDS2017/balanced_binary/'  # Update this path as necessary
normal_model_path = './path/to/save/normal_pytorch_model.pth'  # Update this path as necessary
qat_model_path = './path/to/save/qat_pytorch_model.pth'  # Update this path as necessary


Function to load data

In [3]:
def load_data(data_dir):
    train_data = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    test_data = pd.read_csv(os.path.join(data_dir, 'test.csv'))
    val_data = pd.read_csv(os.path.join(data_dir, 'validation.csv'))
    
    # Drop the ID column
    train_data = train_data.drop(columns=['ID'])
    test_data = test_data.drop(columns=['ID'])
    val_data = val_data.drop(columns=['ID'])
    
    # Encode labels
    label_encoder = LabelEncoder()
    train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
    test_data['Label'] = label_encoder.transform(test_data['Label'])
    val_data['Label'] = label_encoder.transform(val_data['Label'])
    
    return train_data, test_data, val_data, label_encoder

Load data

In [4]:
train_data, test_data, val_data, label_encoder = load_data(data_dir)

# Convert to TabularDataset
train_data_tab = TabularDataset(train_data)
test_data_tab = TabularDataset(test_data)
val_data_tab = TabularDataset(val_data)

Load the models

In [5]:
# Load AutoGluon model
predictor = TabularPredictor.load(predictor_path)

# Load PyTorch model
class AutoReplicatedNN(nn.Module):
    def __init__(self, architecture, input_feature_size):
        super(AutoReplicatedNN, self).__init__()
        layers = []
        current_input_size = input_feature_size
        for layer_type, layer_obj in architecture:
            if layer_type == nn.BatchNorm1d:
                layers.append(nn.BatchNorm1d(current_input_size))
            elif layer_type == nn.Linear:
                layers.append(nn.Linear(current_input_size, layer_obj.out_features))
                current_input_size = layer_obj.out_features
            elif layer_type == nn.ReLU:
                layers.append(nn.ReLU())
            elif layer_type == nn.Dropout:
                layers.append(nn.Dropout(p=layer_obj.p))
            elif layer_type == nn.Softmax:
                layers.append(nn.Softmax(dim=layer_obj.dim))
            else:
                raise ValueError(f"Unhandled layer type: {layer_type}")
        self.main_block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.main_block(x)

def get_model_architecture(predictor, input_feature_size):
    best_model_name = predictor.get_model_best()
    best_model = predictor._trainer.load_model(best_model_name)
    
    architecture = []
    for name, layer in best_model.model.named_children():
        if isinstance(layer, nn.Sequential):
            for sub_layer in layer:
                architecture.append((type(sub_layer), sub_layer))
        else:
            architecture.append((type(layer), layer))
    
    return architecture, best_model, input_feature_size

# Determine the input feature size
input_feature_size = train_data.drop(columns=['Label']).shape[1]

# Get model architecture
architecture, best_model, input_feature_size = get_model_architecture(predictor, input_feature_size)

# Initialize the replicated models
normal_model = AutoReplicatedNN(architecture, input_feature_size)
qat_model = AutoReplicatedNN(architecture, input_feature_size)

# Load the state dictionary of the normal PyTorch model
normal_model.load_state_dict(torch.load(normal_model_path))

# Load the state dictionary of the QAT PyTorch model
qat_model.load_state_dict(torch.load(qat_model_path))


This means that the predictor was fit in an AutoGluon version `<=0.3.1`.


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/predictor/predictor.pkl'

Evaluate models

In [ ]:
# Prepare test data
X_test = test_data.drop(columns=['Label']).values
y_test = test_data['Label'].values

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Evaluate AutoGluon model
ag_predictions = predictor.predict_proba(test_data_tab)
ag_accuracy = accuracy_score(y_test, ag_predictions.idxmax(axis=1))
ag_loss = log_loss(y_test, ag_predictions)
print(f'AutoGluon Model Accuracy: {ag_accuracy:.4f}')
print(f'AutoGluon Model Log Loss: {ag_loss:.4f}')

# Evaluate normal PyTorch model
normal_model.eval()
with torch.no_grad():
    normal_outputs = normal_model(X_test_tensor)
    normal_predictions = torch.argmax(normal_outputs, dim=1)
    normal_accuracy = accuracy_score(y_test_tensor, normal_predictions)
    normal_loss = log_loss(y_test, nn.Softmax(dim=1)(normal_outputs).numpy())
print(f'Normal PyTorch Model Accuracy: {normal_accuracy:.4f}')
print(f'Normal PyTorch Model Log Loss: {normal_loss:.4f}')

# Evaluate QAT PyTorch model
qat_model.eval()
with torch.no_grad():
    qat_outputs = qat_model(X_test_tensor)
    qat_predictions = torch.argmax(qat_outputs, dim=1)
    qat_accuracy = accuracy_score(y_test_tensor, qat_predictions)
    qat_loss = log_loss(y_test, nn.Softmax(dim=1)(qat_outputs).numpy())
print(f'QAT PyTorch Model Accuracy: {qat_accuracy:.4f}')
print(f'QAT PyTorch Model Log Loss: {qat_loss:.4f}')

# Compare model sizes
ag_model_size = os.path.getsize(os.path.join(predictor.path, 'model.pkl'))
normal_model_size = os.path.getsize(normal_model_path)
qat_model_size = os.path.getsize(qat_model_path)
print(f'AutoGluon Model Size: {ag_model_size / 1024:.2f} KB')
print(f'Normal PyTorch Model Size: {normal_model_size / 1024:.2f} KB')
print(f'QAT PyTorch Model Size: {qat_model_size / 1024:.2f} KB')


Plot everything

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Accuracy comparison
axes[0, 0].bar(['AutoGluon', 'Normal PyTorch', 'QAT PyTorch'], [ag_accuracy, normal_accuracy, qat_accuracy], color=['blue', 'orange', 'green'])
axes[0, 0].set_title('Model Accuracy')
axes[0, 0].set_ylabel('Accuracy')

# Log Loss comparison
axes[0, 1].bar(['AutoGluon', 'Normal PyTorch', 'QAT PyTorch'], [ag_loss, normal_loss, qat_loss], color=['blue', 'orange', 'green'])
axes[0, 1].set_title('Model Log Loss')
axes[0, 1].set_ylabel('Log Loss')

# Model size comparison
axes[1, 0].bar(['AutoGluon', 'Normal PyTorch', 'QAT PyTorch'], [ag_model_size / 1024, normal_model_size / 1024, qat_model_size / 1024], color=['blue', 'orange', 'green'])
axes[1, 0].set_title('Model Size')
axes[1, 0].set_ylabel('Size (KB)')

# Complexity comparison (number of parameters)
ag_params = sum(p.numel() for p in best_model.model.parameters())
normal_params = sum(p.numel() for p in normal_model.parameters())
qat_params = sum(p.numel() for p in qat_model.parameters())
axes[1, 1].bar(['AutoGluon', 'Normal PyTorch', 'QAT PyTorch'], [ag_params, normal_params, qat_params], color=['blue', 'orange', 'green'])
axes[1, 1].set_title('Model Complexity')
axes[1, 1].set_ylabel('Number of Parameters')

plt.tight_layout()
plt.show()
